<a href="https://colab.research.google.com/github/AnuragPaul0/IVP/blob/main/pyt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
url = 'https://raw.githubusercontent.com/kmouts/FootAndBall/master/soccernet_v3.yaml'
# 'https://github.com/kmouts/FootAndBall/blob/master/soccernet_v3.yaml'
# 'https://github.com/kmouts/FootAndBall/blob/master/training_stats_model_20230211_1550.pickle'
# 'https://github.com/kmouts/FootAndBall/blob/master/yolov8n.pt'

In [12]:
import os

In [13]:
y = os.path.basename(url); y

'soccernet_v3.yaml'

In [15]:
import urllib.request

In [24]:
with urllib.request.urlopen(url) as file, open(y, 'w') as f:
    f.write(file.read().decode())

In [18]:
import requests

In [19]:
r = requests.get(url, allow_redirects=True)

In [20]:
with open(y, "w") as f:
    f.write(r.text)

In [21]:
urllib.request.urlretrieve(url, y)

('soccernet_v3.yaml', <http.client.HTTPMessage at 0x7e146fff4b50>)

In [22]:
open(y, 'wb').write(r.content)

148582

In [ ]:
pip install ultralytics # 1:19 s, 6

In [7]:
from ultralytics.models.yolo.detect import DetectionTrainer

In [3]:
from ultralytics import YOLO

In [ ]:
from ultralytics.yolo.v8.detect import DetectionTrainer

In [5]:
DetectionTrainer

NameError: name 'DetectionTrainer' is not defined

In [26]:
trainer = DetectionTrainer(overrides={"data": "soccernet_v3.yaml", "imgsz": 1280,
                                      "batch": -1, "workers": 12,
                                      "pretrained": True, "epochs": 100, "patience": 5,
                                      "model": "yolov8n.pt",
                                      "val": True, "device": 0, "name": "yolo8n_custom_1280_SGD",
                                      "rect": True, "verbose": True, "cache": False,
                                      "optimizer": "SGD"
                                      }
                          #  , device=[0,1,2,3]
                           )
trainer.train()
trained_model = trainer.best

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=soccernet_v3.yaml, epochs=100, time=None, patience=5, batch=-1, imgsz=1280, save=True, save_period=-1, cache=False, device=0, workers=12, project=None, name=yolo8n_custom_1280_SGD2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

RuntimeError: Dataset 'soccernet_v3.yaml' error ❌ 
Dataset 'soccernet_v3.yaml' images not found ⚠️, missing path '/mnt/DATA/DATASETS/SOCCERNETv3/SNV3/YOLO/valid/images'
Note dataset download directory is '/content/datasets'. You can update this in '/root/.config/Ultralytics/settings.yaml'

In [ ]:
def train(params: Params):
    if not os.path.exists(MODEL_FOLDER):
        os.mkdir(MODEL_FOLDER)

    assert os.path.exists(MODEL_FOLDER), ' Cannot create folder to save trained model: {}'.format(MODEL_FOLDER)

    dataloaders = make_dataloaders(params)
    print('Training set: Dataset size: {}'.format(len(dataloaders['train'].dataset)))
    if 'val' in dataloaders:
        print('Validation set: Dataset size: {}'.format(len(dataloaders['val'].dataset)))

    # Create model
    device = "cuda" if torch.cuda.is_available() else 'cpu'
    model = footandball.model_factory(params.model, 'train')
    model.print_summary(show_architecture=True)
    model = model.to(device)

    model_name = 'model_' + time.strftime("%Y%m%d_%H%M")
    print('Model name: {}'.format(model_name))

    optimizer = optim.Adam(model.parameters(), lr=params.lr)
    scheduler_milestones = [int(params.epochs * 0.75)]
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, scheduler_milestones, gamma=0.1)
    train_model(model, optimizer, scheduler, params.epochs, dataloaders, device, model_name)


if __name__ == '__main__':
    print('Train FoootAndBall detector on ISSIA dataset')
    parser = argparse.ArgumentParser()
    parser.add_argument('--config', help='Path to the configuration file', type=str, default='config.txt')
    parser.add_argument('--debug', dest='debug', help='debug mode', action='store_true')
    args = parser.parse_args()

    print('Config path: {}'.format(args.config))
    print('Debug mode: {}'.format(args.debug))

    params = Params(args.config)
    params.print()

    train(params)

In [ ]:
def train_model(model, optimizer, scheduler, num_epochs, dataloaders, device, model_name):
    # Weight for components of the loss function.
    # Ball-related loss and player-related loss are mean losses (loss per one positive example)
    alpha_l_player = 0.01
    alpha_c_player = 1.
    alpha_c_ball = 5.

    # Normalize weights
    total = alpha_l_player + alpha_c_player + alpha_c_ball
    alpha_l_player /= total
    alpha_c_player /= total
    alpha_c_ball /= total

    # Loss function
    criterion = SSDLoss(neg_pos_ratio=3)

    is_validation_set = 'val' in dataloaders
    if is_validation_set:
        phases = ['train', 'val']
    else:
        phases = ['train']

    # Training statistics
    training_stats = {'train': [], 'val': []}

    print('Training...')
    for epoch in tqdm.tqdm(range(num_epochs)):
        # Each epoch has a training and validation phase
        # for phase in ['train', 'val']:
        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            batch_stats = {'loss': [], 'loss_ball_c': [], 'loss_player_c': [], 'loss_player_l': []}

            count_batches = 0
            # Iterate over data.
            for ndx, (images, boxes, labels) in enumerate(dataloaders[phase]):
                images = images.to(device)
                h, w = images.shape[-2], images.shape[-1]
                gt_maps = model.groundtruth_maps(boxes, labels, (h, w))
                gt_maps = [e.to(device) for e in gt_maps]
                count_batches += 1

                with torch.set_grad_enabled(phase == 'train'):
                    predictions = model(images)
                    # Backpropagation
                    optimizer.zero_grad()
                    loss_l_player, loss_c_player, loss_c_ball = criterion(predictions, gt_maps)

                    loss = alpha_l_player * loss_l_player + alpha_c_player * loss_c_player + alpha_c_ball * loss_c_ball

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                count_batches += 1
                batch_stats['loss'].append(loss.item())
                batch_stats['loss_ball_c'].append(loss_c_ball.item())
                batch_stats['loss_player_c'].append(loss_c_player.item())
                batch_stats['loss_player_l'].append(loss_l_player.item())

            # Average stats per batch
            avg_batch_stats = {}
            for e in batch_stats:
                avg_batch_stats[e] = np.mean(batch_stats[e])

            training_stats[phase].append(avg_batch_stats)
            s = '{} Avg. loss total / ball conf. / player conf. / player loc.: {:.4f} / {:.4f} / {:.4f} / {:.4f}'
            print(s.format(phase, avg_batch_stats['loss'], avg_batch_stats['loss_ball_c'],
                           avg_batch_stats['loss_player_c'], avg_batch_stats['loss_player_l']))

        # Scheduler step
        scheduler.step()
        print('')

    model_filepath = os.path.join(MODEL_FOLDER, model_name + '_final' + '.pth')
    torch.save(model.state_dict(), model_filepath)

    with open('training_stats_{}.pickle'.format(model_name), 'wb') as handle:
        pickle.dump(training_stats, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return training_stats

# Pickle

In [41]:
import pickle

In [58]:
# y = 'training_stats_model_20230209_1818.pickle'; # 150
y = 'training_stats_model_20230203_2106.pickle'
with open(y, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it. https://docs.python.org/3/library/pickle.html
    data = pickle.load(f)
type(data),len(data)

(dict, 2)

In [ ]:
data['train']

In [60]:
type(data['train']),len(data['train'])

(list, 150)

In [59]:
data

{'train': [{'loss': 2.223653394830895,
   'loss_ball_c': 2.2155314550250576,
   'loss_player_c': 2.286158445450287,
   'loss_player_l': 0.034160682560591876},
  {'loss': 2.188464044125792,
   'loss_ball_c': 2.1788429507531584,
   'loss_player_c': 2.25804651514026,
   'loss_player_l': 0.04080402739273173},
  {'loss': 2.188137274522048,
   'loss_ball_c': 2.179347601210434,
   'loss_player_c': 2.2535260606464846,
   'loss_player_l': 0.04413640437492278},
  {'loss': 2.1865120474953383,
   'loss_ball_c': 2.177759777302848,
   'loss_player_c': 2.251694843697579,
   'loss_player_l': 0.044405976189555406},
  {'loss': 2.1856681144221977,
   'loss_ball_c': 2.176845533440601,
   'loss_player_c': 2.25120028523143,
   'loss_player_l': 0.04378024164849535},
  {'loss': 2.1853904980568717,
   'loss_ball_c': 2.176808007537464,
   'loss_player_c': 2.249712439217499,
   'loss_player_l': 0.04448134226923641},
  {'loss': 2.1854687141159834,
   'loss_ball_c': 2.1769698749465145,
   'loss_player_c': 2.249370

In [10]:
import torch

In [25]:
model = torch.load(y)

In [27]:
print(type(model))

<class 'dict'>


In [28]:
for k in model: print(k)

epoch
best_fitness
model
ema
updates
optimizer
train_args
date
version


In [34]:
w = 'train_args'
# model'
# epoch'
for k in model[w]: print(w, k)

train_args task
train_args mode
train_args model
train_args data
train_args epochs
train_args patience
train_args batch
train_args imgsz
train_args save
train_args cache
train_args device
train_args workers
train_args project
train_args name
train_args exist_ok
train_args pretrained
train_args optimizer
train_args verbose
train_args seed
train_args deterministic
train_args single_cls
train_args image_weights
train_args rect
train_args cos_lr
train_args close_mosaic
train_args resume
train_args overlap_mask
train_args mask_ratio
train_args dropout
train_args val
train_args save_json
train_args save_hybrid
train_args conf
train_args iou
train_args max_det
train_args half
train_args dnn
train_args plots
train_args source
train_args show
train_args save_txt
train_args save_conf
train_args save_crop
train_args hide_labels
train_args hide_conf
train_args vid_stride
train_args line_thickness
train_args visualize
train_args augment
train_args agnostic_nms
train_args retina_masks
train_args for

In [26]:
model.eval()

AttributeError: 'dict' object has no attribute 'eval'